# Chapter 3 - Text Matching

## Text Matching Techniques

In [ ]:
%pip install jellyfish
import jellyfish as jf

In [ ]:
jf.levenshtein_distance('Michael','Micheal')

In [ ]:
min(jf.levenshtein_distance('el','ael'), jf.levenshtein_distance('ael','al'), jf.levenshtein_distance('el','al'))

In [ ]:
jf.jaro_similarity('Michael','Micheal')

In [ ]:
jf.jaro_winkler_similarity('Michael','Micheal')

In [ ]:
jf.jaro_winkler_similarity('michael','MICHAEL')

In [ ]:
jf.jaro_winkler_similarity('michael'.lower(),'MICHAEL'.lower())

In [ ]:
jf.metaphone('michael')

In [ ]:
jf.metaphone('micheal')

In [ ]:
import pandas as pd

In [ ]:
mylist = ['Michael','Micheal','Michel','Mike','Mick']

In [ ]:
combs = []
import itertools
for a, b in itertools.combinations(mylist, 2):
    combs.append([a,b,jf.jaro_similarity(a,b),jf.jaro_winkler_similarity(a, b), jf.levenshtein_distance(a,b), jf.match_rating_comparison(a,b),(jf.soundex(a)==jf.soundex(b))])
pd.DataFrame(combs, columns=['Name1','Name2','Jaro','JaroW','Levenshtein','MRA','Soundex'])

## Sample Problem

In [ ]:
df_w_un = pd.read_csv('mps_unmatched.csv')
df_w_un

In [ ]:
df_w_un['Firstname_jaro'] = df_w_un.apply(lambda x: jf.jaro_winkler_similarity(x.Firstname_w, x.Firstname_t), axis=1)
df_w_un['Lastname_jaro'] = df_w_un.apply(lambda x: jf.jaro_winkler_similarity(x.Lastname_w, x.Lastname_t), axis=1)

In [ ]:
df_w_un[(df_w_un['Firstname_jaro'] > 0.8) & (df_w_un['Lastname_jaro'] > 0.8)]

In [ ]:
import pandas as pd

df_w = pd.read_csv('mps_wiki_clean.csv')
df_t = pd.read_csv('mps_they_clean.csv')

cross = df_w.merge(df_t, how='cross',suffixes=('_w', '_t'))

In [ ]:
cross.head(n=5)

In [ ]:
cross['Firstname_jaro'] = cross.apply(lambda x: True if jf.jaro_winkler_similarity(x.Firstname_w, x.Firstname_t)>0.8 else False, axis=1)
cross['Lastname_jaro'] = cross.apply(lambda x: True if jf.jaro_winkler_similarity(x.Lastname_w, x.Lastname_t)>0.8 else False, axis=1)

In [ ]:
cross.head(n=5)

In [ ]:
tp = cross[(cross['Firstname_jaro'] & cross['Lastname_jaro']) & (cross['Constituency_w']==cross['Constituency_t'])]
len(tp)

In [ ]:
fp = cross[(cross['Firstname_jaro'] & cross['Lastname_jaro']) & (cross['Constituency_w']!=cross['Constituency_t'])]
len(fp)

In [ ]:
fp.head(n=5)

In [ ]:
fntn = cross[(~cross['Firstname_jaro'] | ~cross['Lastname_jaro']) & (cross['Constituency_w']==cross['Constituency_t'])]
len(fntn)

In [ ]:
fntn

In [ ]:
df_w['Firstname'].value_counts().mean()

In [ ]:
df_w['Lastname'].value_counts().mean()

In [ ]:
df_w['Constituency'].value_counts().mean()

In [ ]:
df_w_un['Firstname_w_meta'] = df_w_un.apply(lambda x: jf.metaphone(x.Firstname_w), axis=1)

In [ ]:
df_w_un

In [ ]:
df_w_un['Firstname_t_meta'] = df_w_un.apply(lambda x: jf.metaphone(x.Firstname_t), axis=1)

In [ ]:
df_w_un

In [ ]:
df_w_un['Lastname_w_meta'] = df_w_un.apply(lambda x: jf.metaphone(x.Lastname_w), axis=1)
df_w_un['Lastname_t_meta'] = df_w_un.apply(lambda x: jf.metaphone(x.Lastname_t), axis=1)

In [ ]:
df_w_un

In [ ]:
combs = []
import itertools
for a, b in itertools.combinations(mylist, 2):
    combs.append([a,b,jf.jaro_similarity(a,b),jf.jaro_winkler_similarity(a, b), jf.levenshtein_distance(a,b), jf.match_rating_comparison(a,b),(jf.soundex(a)==jf.soundex(b)), jf.metaphone(a), jf.metaphone(b)])
pd.DataFrame(combs, columns=['Name1','Name2','Jaro','JaroW','Levenshtein','MRA','Soundex','Name1Meta','Name2Meta'])